# 使用 milvus 管理文档

## 准备

In [10]:
%%time

from llama_index.core import SummaryIndex, Document
from llama_index.core import Settings

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

from milvus import default_server
from pymilvus import connections, utility

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core.llms import ChatMessage

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 25 µs


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fecb5ea4f10>, id_func=<function default_id_func at 0x7fed7ba8f010>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

# 启动milvus服务器

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

# Stop your milvus server
# default_server.stop()

default_server.listen_port

v2.3.5-lite
CPU times: user 3.16 s, sys: 284 ms, total: 3.45 s
Wall time: 6.66 s


19530

In [4]:
%%time

# 初始化全局 llm

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 114 ms, sys: 7.87 ms, total: 122 ms
Wall time: 121 ms


In [5]:
%%time

# 初始化全局 embedding 模型

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 396 ms, sys: 12.3 ms, total: 408 ms
Wall time: 407 ms


In [6]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 352 µs, sys: 0 ns, total: 352 µs
Wall time: 356 µs


In [7]:
%%time

# load documents
documents = SimpleDirectoryReader("./books1/").load_data()
for doc in documents:
    print(doc.doc_id, doc.metadata['file_name'])

5ef51c3f-08e9-4ec4-a409-9ee797789c40 孔乙己.txt
5b3fe221-0637-4660-8ea8-fe0e73f4edfc 社戏.txt
CPU times: user 9.54 ms, sys: 0 ns, total: 9.54 ms
Wall time: 9.11 ms


In [8]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: d39afef27ad244c58d5642f568635721
Created new connection using: d39afef27ad244c58d5642f568635721
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection
Successfully created an index on collection: llamacollection
CPU times: user 746 ms, sys: 37 ms, total: 783 ms
Wall time: 14.5 s


## 直接用llm对话

In [11]:
%%time

gen = llm.stream_chat([ChatMessage(role="user", content="介绍下孔乙己")])
for response in gen:
    print(response.delta, end="", flush=True)

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生在小说《呐喊》中塑造的一个经典人物形象。他是19世纪末中国旧社会的一个底层知识分子，以“孔乙己”这一绰号为人所知。孔乙己生活贫困，穿着破旧的长衫，常去咸亨酒店喝酒，但他对自己的身份和学问颇为自豪，尽管他连基本的生活技能都不具备，比如识字、写字。

孔乙己的故事反映了那个时代的社会冷漠和阶级差距，人们对他既嘲笑又同情。他的悲剧命运在于他无法适应社会的变化，也无法融入普通人的生活，最终在贫困和嘲笑中默默死去。这个角色成为了中国文学中一个深刻的社会寓言，提醒人们关注底层人民的生存状态和人性的复杂性。CPU times: user 271 ms, sys: 22.7 ms, total: 294 ms
Wall time: 8.81 s


## 使用嵌入的问答

In [12]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.16

## rerank

In [13]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 1.09 s, sys: 677 ms, total: 1.76 s
Wall time: 963 ms


In [14]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生笔下《呐喊》中的人物，他是一个独特的形象。他以站着喝酒而穿长衫的装束在鲁镇的酒店中显得与众不同，这体现了他的社会地位和经济状况可能较为特殊，可能是穷困的知识分子。他身材高大，脸色青白且带有伤痕，留着乱蓬蓬的花白胡子，给人一种落魄但又坚韧的感觉。

孔乙己的生活并不如意，他的存在为周围的人带来了一丝欢乐，因为掌柜和主顾们对他有些同情，他的笑声能打破店内的沉闷。然而，他似乎并未完全融入这个社会，与其他人有着明显的隔阂，比如他无法与他们谈天，只能与孩子们交流。

酒店的环境也反映了孔乙己的生活状态，当街的曲尺形柜台和随时温酒的服务，暗示了他可能是个常客，但他的穿着破旧，如盘腿而坐、用草绳挂蒲包等细节，显示出他的贫困。他对学问有些许自豪，试图通过考问我这样的孩子来证明自己的身份或寻求一丝尊严。

总的来说，孔乙己是一个被社会边缘化，既可怜又可悲的知识分子形象，他的故事揭示了当时社会的冷漠和不公。
CPU times: user 1.86 s, sys: 105 ms, total: 1.96 s
Wall time: 12.5 s


## 删除社戏后的嵌入问答

In [15]:
%%time

index.delete_ref_doc("5b3fe221-0637-4660-8ea8-fe0e73f4edfc", delete_from_docstore=True)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生笔下的一位经典人物，他生活在清朝末年的鲁镇，以读书人的身份自居，但实际上没有功名，连秀才都不是，这让他在社会上处于边缘地位。他的穿着特征是破旧的长衫，脸上有伤疤，这是他被嘲笑和歧视的一个标志。孔乙己常去咸亨酒店喝酒，茴香豆是他喜欢的小吃，但他经济拮据，常常赊账，甚至偶尔偷窃以维持生计。他的生活充满了困苦和无奈，他的遭遇揭示了当时社会的冷漠和底层人民的艰难生活状态。尽管他试图通过学习写字来提升自己，但最终因为偷窃被打断腿，这标志着他人生的彻底衰落。孔乙己的故事不仅是个体的悲剧，也是对那个时代社会问题的深刻反映。|>
CPU times: user 394 ms, sys: 42.6 ms, total: 437 ms
Wall time: 18.4 s


## 删除社戏后的rerank问答

In [16]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生笔下《呐喊》中的人物，他是一个独特的形象。他以站着喝酒而穿长衫的装束在鲁镇的酒店中出现，这表明了他的社会地位不高，可能是个落魄的读书人。他的身材高大，脸色青白且带有伤痕，胡子乱蓬蓬的，显示出生活的艰辛和岁月的痕迹。

孔乙己的生活并不如意，他的存在为周围环境带来了一丝欢乐，因为掌柜和主顾们对他有些同情，他的笑声能打破店内的沉闷。酒店的格局是当街的曲尺形大柜台，这可能是他常去的地方，尽管他可能没有什么失职，但工作单调无聊，环境压抑。

孔乙己的形象刻画得非常细致，从他的穿着（破夹袄、蒲包和草绳）可以看出他的贫困，而他对孩子的提问则显示出他对知识的渴望，尽管他自己可能并不受人尊重。总的来说，孔乙己是一个被社会边缘化，但又保持着尊严和知识追求的人物。
CPU times: user 741 ms, sys: 22.1 ms, total: 763 ms
Wall time: 9.53 s


## 继续删除孔乙己文档后嵌入问答

In [17]:
%%time

index.delete_ref_doc("5ef51c3f-08e9-4ec4-a409-9ee797789c40", delete_from_docstore=True)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 25.6 ms, sys: 4.05 ms, total: 29.6 ms
Wall time: 719 ms


## 删除孔乙己文档后rerank问答

In [18]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

Empty Response
CPU times: user 8.06 ms, sys: 0 ns, total: 8.06 ms
Wall time: 267 ms


## 重新插入孔乙己文档后的嵌入

In [21]:
%%time

documents = SimpleDirectoryReader(input_files=["books1/孔乙己.txt"]).load_data()
for doc in documents:
    print(doc.doc_id, doc.metadata['file_name'])

index.insert(documents[0])

4edc2387-1f2c-4a7e-96fb-70894dc7a215 孔乙己.txt
CPU times: user 213 ms, sys: 21.6 ms, total: 235 ms
Wall time: 4.57 s


In [23]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生在《孔乙己》这篇小说中塑造的一个典型角色，他生活在清朝末年的鲁镇，以咸亨酒店为常驻地。他穿着破旧的长衫，尽管经济拮据，却保持着读书人的自尊，喜欢谈论古文和一些似通非通的典故，如“君子固穷”和“者乎”等，这反映了他试图保持文化和知识上的优越感。他的主要活动是温酒，但其实并不真正参与酒店的日常经营，掌柜认为他不适合侍候穿长衫的主顾。

孔乙己的生活充满了贫困和孤独，他用四文大钱买酒，手满是泥，这暗示了他生活的艰辛。他的形象揭示了当时社会对底层人民的冷漠态度，以及科举制度下读书人地位的尴尬。小说通过他的命运变迁，探讨了社会的残酷和人性的复杂。尽管他的结局悲惨，但他的故事成为了文学作品中一个深刻的社会寓言。
CPU times: user 412 ms, sys: 37.6 ms, total: 450 ms
Wall time: 18.8 s


## 重新插入孔乙己文档后的rerank

In [24]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生笔下《呐喊》中的人物，他是个独特的形象，穿着长衫却站着喝酒，这在当时的社会中显得格格不入。他有学问，曾读过书但未能考取秀才（进学），因此生活贫困，只能通过抄写书籍来换取食物。孔乙己说话喜欢用文言文，如“之乎者也”，这让他的话半懂不懂，人们根据他的姓氏和描红纸上的“上大人孔乙己”给他取了绰号。

他的性格中既有善良的一面，能给周围的人带来欢乐，但也有缺点，好吃懒做，导致他经济状况每况愈下。由于偷窃的行为，他失去了抄书的工作，生活更加艰难。总的来说，孔乙己是一个穷困潦倒、边缘化的知识分子形象，反映了当时社会的冷漠和不公。
CPU times: user 582 ms, sys: 18.4 ms, total: 600 ms
Wall time: 8.13 s


## 尝试并发问答模式

In [25]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5,
    response_mode="tree_summarize" 
)

streaming_response = query_engine.query("介绍下孔乙己")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孔乙己是鲁迅先生在短篇小说《呐喊》中塑造的一个经典人物，他生活在中国清朝末年的社会底层。孔乙己以读书人自居，却穿着破旧的长衫，这象征着他的社会地位不高，被归类为“短衣帮”。他常去鲁镇的咸亨酒店赊账喝酒，尽管经济困难，但他坚持这种生活方式，体现了他对传统文化和知识的坚守。

孔乙己曾经有过进学的经历，是个秀才，但科举制度并未改变他的贫困命运。他通过谈论“回”字和引用“君子固穷”的典故来展现自己的文化素养，然而这在现实生活中并没有给他带来实质的帮助，反而成为他人嘲笑和捉弄的对象。

孔乙己的故事揭示了当时社会的冷漠无情，以及读书人地位的尴尬。他的悲剧命运反映了科举制度下的知识分子困境，鲁迅通过这个角色批判了旧式教育和封建社会的不公。孔乙己的形象在鲁迅笔下具有深刻的讽刺意味和社会批判价值。
CPU times: user 337 ms, sys: 41.6 ms, total: 378 ms
Wall time: 29.4 s
